In [30]:
import os

# import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display
# import get_related_artists as from_echo
import re
import pickle
from IPython.display import display, HTML
import python_utils as utils


In [31]:
# Load metadata and features.
tracks = pd.read_csv('fma_metadata/tracks.csv')
r_tracks = pd.read_csv('fma_metadata/raw_tracks.csv')
genres = pd.read_csv('fma_metadata/genres.csv')
features = pd.read_csv('fma_metadata/features.csv')
echonest = pd.read_csv('fma_metadata/echonest.csv')
r_artists = pd.read_csv('fma_metadata/raw_artists.csv')

tracks.shape, genres.shape, features.shape, echonest.shape

((106576, 53), (163, 5), (106577, 519), (13132, 250))

In [32]:
tracks_sm = tracks.loc[(tracks['set.1'] == 'small') & (tracks['set'] == 'training')]

In [33]:
# Spotify  Business

import sys
import spotipy
import pprint
''' shows recommendations for the given artist
'''


from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id='ba6790bdcd434f06b7b577e344c6e0ae', client_secret='145715e565ff48469c306484896a34f5')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None


def get_related(uri):
    
    related = sp.artist_related_artists(uri)
    ans = [] 
    
    for artist in related['artists']:
        ans.append(artist['name'])
#        print artist['name']
        
    
    return ans
    
def save_spot_rel_artists(name, table):
    with open('%s.pickle'%name, 'wb') as writer:
        pickle.dump(table, writer, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_spot_rel_artists(name):
    with open(name, 'rb') as handle:
        thing = pickle.load(handle)   
    return thing

In [6]:
# get spotify artist_id for every artist, and related artists
na = 0
artist_rel = {}
artists_sm = tracks_sm['artist.12'].unique()

for i, elem in enumerate(artists_sm):
    
    artist = re.split(',', elem)[0]
    if i%100 == 0:
        print i
    
#     a_id = from_echo.get_artist(artist)
    try:
        a_id = get_artist(artist)

        if a_id:
    #         related_artists = from_echo.get_related(a_id['uri'])
            related_artists = get_related(a_id['uri'])
    #        print related_artists
            if related_artists:
                artist_rel[artist] = related_artists
    #             artist_rel[artist] = a_id
            else:
                na += 1
        else:
            na += 1
    except:
        continue
        



0


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/anirudhmani/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Users/anirudhmani/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/Users/anirudhmani/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/Users/anirudhmani/anaconda2/lib/python2.7/inspect.py", line 1051, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/Users/anirudhmani/anaconda2/lib/python2.7/inspect.py", line 1011, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/Users/anirudhmani/anaconda2/lib/python2.7/inspect.py", line 453, in getsourcefile
    if hasattr(getmodule(obje

IndexError: string index out of range

In [34]:
# save_spot_rel_artists("artist_rel_medium_training", artist_rel)
artist_rel = load_spot_rel_artists("artist_rel_small_training.pickle")


In [35]:
len(artist_rel)

1026

In [36]:
# Subsetting Business, Creating Master tables

# Creating Condensed Artist Map, and Master 2D Table
artist_all = {}

for key in artist_rel:
    artist_all[key] = set()

for key in artist_rel:
    for elem in artist_rel[key]:
        if artist_rel.get(elem, None) is not None:
            artist_all[key].add(elem)
            artist_all[elem].add(key)

# assert len(artist_all) == len(artist_rel)  
# deleting artists with no related artist in set
keys = artist_all.keys()
for key in keys:
    if not artist_all[key]:
        del artist_all[key]
#         pass
    else:
        artist_all[key] = list(artist_all[key])
        
# Artist Index Map
artist_index = {}
for i, a in enumerate(artist_all.keys()):
    artist_index[a] = i


In [37]:
artist_rel_master = np.zeros((len(artist_all.keys()), len(artist_all.keys())))

# Master table of artist relation
for i, a in enumerate(artist_all.keys()):
    # going through each artist
    for b in artist_all[a]:
        artist_rel_master[artist_index[a]][artist_index[b]] = 1

# print artist_rel_master[artist_index['Robo']][artist_index['Robo']]

In [38]:
# Master table of artists per genre

# songs per genre
genres_all = {} # maybe redundant
c = 0
artists_set = list(tracks_sm['artist.12'])
genres_artist = {}
i = 0
for gens in tracks_sm['track.9']:
    l = [re.split(']', s) for s in re.split(', |\[', gens)]
    gens = [item for sublist in l for item in sublist]
    for g in gens:
        if g.isdigit():
            if genres_all.get(int(g), -1) != -1:
                pass
            else:
                genres_all[int(g)] = c
                c += 1
            if genres_artist.get(int(g), -1) == -1:
                genres_artist[int(g)] = set()
            if artist_all.get(artists_set[i], None) is not None:
                genres_artist[int(g)].add(artists_set[i]) 
    i += 1



In [39]:
# creating master table of artists per genre

genre_artist_master = np.zeros((len(genres_all.keys()), len(artist_all.keys())))

for g in genres_all.keys():
#     print genres_artist[g]
    for a in genres_artist[g]:
#         print a
        genre_artist_master[genres_all[g]][artist_index[a]] = 1
        
# print genre_artist_master

In [40]:
# Number of songs per artist
songs_per_artist = {}
for k in artist_all.keys():
    songs_per_artist[k] = len(tracks_sm[tracks_sm['artist.12'] == k])

In [41]:
# Calculating D
def calc_D(songs_per_artist, num_artists_set):

    # total number of songs in Set
    numer = 0
    num_songs_set = 0
    for k in songs_per_artist:
        num_songs_set += songs_per_artist[k]
        if songs_per_artist[k] > 0.0:
            numer += np.log(1.0 - (1.0/(1 + songs_per_artist[k])))
           
    D = numer/num_songs_set
    return D

In [42]:
# Calculating C
def calc_C(num_artists_set, genres_artist):

    # total number of genres in Set
    sum_gen = genres_artist.sum(axis=1)
    num_gen_set = np.count_nonzero(sum_gen)
    numer = 0.
    for k in range(len(genres_artist)):
        if sum_gen[k] > 0.0:
            numer += np.log(1.0 - (1.0/(1 + sum_gen[k])))
    
    C = numer/len(sum_gen) 

    return C

In [43]:
# Calculating S
def calc_S(artists_in_set, songs_per_artist):

    num_edges = np.count_nonzero(artists_in_set, axis=1)
    numer = 0.0
    for n_edges in num_edges:
        if n_edges > 0.0:
            numer += np.log(1.0 - (1./(1. + n_edges)))
#         print numer
    S = numer/len(num_edges)
    
    return S

In [52]:
# Main Loop of Calculation
import copy
# N is number of Artists in Set


N = 200    
artists_in_set = np.random.choice(artist_all.keys(), N, replace=False)
artists_not_in_set = artist_all.keys()
for a in artists_in_set:
    artists_not_in_set.remove(a)

len(artists_in_set), len(artists_not_in_set)



(200, 194)

In [53]:
# songs per artist subset    
s_p_a = {}
for a in artists_in_set:
    s_p_a[a] = songs_per_artist[a]

# artists per genre subset
num_genres, num_artists = genre_artist_master.shape
g_a = np.zeros((num_genres, N))
a_rel = np.zeros((N, N))
index = 0

ai = [artist_index[k] for k in artists_in_set]

for i in range(num_genres):
    g_a[i, :] = genre_artist_master[i, ai]

for a in artists_in_set:
    ind = artist_index[a]
    a_rel[index, :] = artist_rel_master[ind, ai]
    a_rel[:, index] = artist_rel_master[ai, ind]    
    index +=1

In [54]:
best_artist = []

C = calc_C(N, g_a)
D = calc_D(s_p_a, N)
S = calc_S(a_rel, s_p_a)
goodness = C + D + S

print goodness

g_list = []
best_set = []
g_list_local = [goodness]
C_list = [C]
D_list = [D]
S_list = [S]


for epoch in range(5):
    

    # main loop for optimization
    for i, a in enumerate(artists_in_set):
        #remove an artist
        s_p_a[a] = 0
        g_a[:, i] = 0
        a_rel[i, :] = 0
        a_rel[:, i] = 0

        change = 0
        local_goodness = - float('inf')
        local_S = local_D = local_C = 0.
        # add artist from other set one at a time
        for b in artists_not_in_set:

            # add from other set
            s_p_a[b] = songs_per_artist[b]
            ai[i] = artist_index[b]
            g_a[:, i] = genre_artist_master[:, artist_index[b]]
            a_rel[i, :] = artist_rel_master[artist_index[b], ai]
            a_rel[:, i] = artist_rel_master[ai, artist_index[b]]
            
            C = calc_C(N, g_a)
            D = calc_D(s_p_a, N)
            S = calc_S(a_rel, s_p_a)

            G = C + D + S
            if G > local_goodness:
                local_goodness = G
                local_C = C
                local_D = D
                local_S = S
                change = b

            # delete other element
            s_p_a[b] = 0
            g_a[:, i] = 0
            a_rel[i, :] = 0
            a_rel[:, i] = 0       

#         print goodness
        if local_goodness > goodness:
            goodness = local_goodness
            C = local_C
            D = local_D
            S = local_S
            ai[i] = artist_index[change]
            s_p_a[change] = songs_per_artist[change]
            g_a[:, i] = genre_artist_master[:, artist_index[change]]
            a_rel[i, :] = artist_rel_master[artist_index[change], ai]
            a_rel[:, i] = artist_rel_master[ai, artist_index[change]]
            artists_not_in_set.append(a)
            artists_not_in_set.remove(change)
        else:
            ai[i] = artist_index[a]
            s_p_a[a] = songs_per_artist[a]
            g_a[:, i] = genre_artist_master[:, artist_index[a]]
            a_rel[i, :] = artist_rel_master[artist_index[a], ai]
            a_rel[:, i] = artist_rel_master[ai, artist_index[a]]
        


        g_list_local.append(goodness)
        C_list.append(C)
        D_list.append(D)
        S_list.append(S)
    
    
    g_list.append(np.mean(np.array(g_list_local), axis=0))
    print epoch, g_list_local[-1], C_list[-1], D_list[-1], S_list[-1]

-0.707849229822
0 -0.299325020859 -0.135872363784 -0.0767427258229 -0.0867345674764
1 -0.299325020859 -0.217448212036 -0.0767427258229 -0.381563706195


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/anirudhmani/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Users/anirudhmani/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/Users/anirudhmani/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/Users/anirudhmani/anaconda2/lib/python2.7/inspect.py", line 1051, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/Users/anirudhmani/anaconda2/lib/python2.7/inspect.py", line 1011, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/Users/anirudhmani/anaconda2/lib/python2.7/inspect.py", line 453, in getsourcefile
    if hasattr(getmodule(obje

IndexError: string index out of range

In [199]:
genre_artist_master.shape

(127, 928)

In [51]:
g_list_local[0]

-0.71966374465951866

In [48]:
def save_goodness(name, table):
    with open('%s.pickle'%name, 'wb') as writer:
        pickle.dump(table, writer, protocol=pickle.HIGHEST_PROTOCOL)
        
save_goodness("C_200", C_list)
save_goodness("D_200", D_list)
save_goodness("S_200", S_list)
save_goodness("G_200", g_list_local)